In [1]:
library(tidyverse)
library(readxl)
library(broom)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data <- read_excel("data/raw_elisa_data.xlsx")
# tidy data
data <- data[-c(1:4, 8, 9, 11, 12, 14, 15, 17)]
colnames(data) <- c("sex", "treatment", "stimulus", "CV1", "CV2", "CV3", "IGFBP3", "IGF1")
data <- data |> 
    filter(CV1<20 & CV2<20 & CV3<30)|> 
    select(-c("CV1", "CV2", "CV3"))|> 
    mutate(treatment = case_when(treatment == "Water" ~ "water",
                                 treatment == "Sucrose" ~ "sucrose", 
                                 TRUE ~ as.character(treatment)))|> 
    mutate(stimulus = case_when(stimulus == "Handle" ~ "handle",
                                stimulus == "Needle" ~ "needle",
                                stimulus == "Pressure" ~ "pressure",
                                TRUE ~ as.character(stimulus)))

In [3]:
head(data)


sex,treatment,stimulus,IGFBP3,IGF1
<chr>,<chr>,<chr>,<dbl>,<dbl>
F,sucrose,pressure,0.0002226082,6.159189e-06
F,sucrose,needle,0.0002600661,5.697212e-06
M,water,needle,0.0002794693,5.294438e-06
F,water,pressure,0.0003398651,5.819291e-06
M,sucrose,handle,0.0002621692,9.358682e-06
M,water,handle,0.0003166086,4.394378e-06


# Sex

In [4]:
# pretty sure sex doesnt matter, but just to check
lm_igf1_sex <- lm(IGF1~ sex, data)
tidy(lm_igf1_sex)


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),6.976059e-06,4.774864e-07,14.6099646,1.888112e-26
sexM,-2.390272e-07,6.855255e-07,-0.3486773,7.280720e-01


In [5]:
lm_igfbp3_sex <- lm(IGFBP3~ sex, data)
tidy(lm_igfbp3_sex)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),2.658054e-04,1.470978e-05,18.06998091,4.087206e-33
sexM,-1.302992e-06,2.111878e-05,-0.06169826,9.509275e-01


In [6]:
data <- data|> 
    select(-sex)

# IGF1

In [7]:
lm_igf1_treat <- lm(IGF1~ treatment, data)
tidy(lm_igf1_treat)
#Sucrose significant?

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),4.270462e-06,1.271009e-06,3.359900,0.00111218
treatmentsucrose,3.175543e-06,1.357066e-06,2.340006,0.02131154
treatmentwater,2.335813e-06,1.368382e-06,1.706989,0.09099070


In [8]:
lm_igf1_stim <- lm(IGF1 ~ stimulus, data)
tidy(lm_igf1_stim) 
#Pressure Significant?

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),4.270462e-06,1.265396e-06,3.374804,0.001063318
stimulushandle,2.401795e-06,1.400997e-06,1.714346,0.089658718
stimulusneedle,2.236757e-06,1.405292e-06,1.591667,0.114713379
stimuluspressure,3.636199e-06,1.393155e-06,2.610047,0.010487759


In [9]:
glance(lm_igf1_stim)

r.squared,adj.r.squared,sigma,statistic,p.value,df,logLik,AIC,BIC,deviance,df.residual,nobs
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
0.07481403,0.04620003,3.347923e-06,2.614596,0.05550757,3,1132.052,-2254.104,-2241.029,1.087233e-09,97,101


In [10]:
glance(lm_igf1_treat)

r.squared,adj.r.squared,sigma,statistic,p.value,df,logLik,AIC,BIC,deviance,df.residual,nobs
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
0.05696512,0.03771951,3.362773e-06,2.959902,0.05647551,2,1131.087,-2254.174,-2243.714,1.108208e-09,98,101


Repeat process for IGFBP3, then use nested model ANOVA to test if interaction models are better than base model (which is equivalent to null model).

# IGFBP3